In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

import config 

                       IMAGE TRANSFORMS

In [2]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [3]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
CUDA version: 11.8
GPU name: NVIDIA GeForce RTX 3050 4GB Laptop GPU


In [4]:
train_transform = transforms.Compose([
    transforms.Resize((config.IMAGE_SIZE, config.IMAGE_SIZE)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transform = transforms.Compose([
    transforms.Resize((config.IMAGE_SIZE, config.IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


DATA LOADING

In [5]:
train_dataset = datasets.ImageFolder(
    config.TRAIN_DIR,
    transform=train_transform
)

train_loader = DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=True,
    num_workers=0,         
    pin_memory=True
)


print("Classes:", train_dataset.classes)
print("Number of training samples:", len(train_dataset))

Classes: ['dyslexic', 'normal']
Number of training samples: 151649


LOADING RESNET-50

In [6]:
model = models.resnet50(pretrained=True)

c:\Users\ASUS\anaconda3\envs\dyslexia_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\anaconda3\envs\dyslexia_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#FREEZING THE BASE MODEL
(to prevent it from the overfitting)

In [7]:
for param in model.parameters():
    param.requires_grad = False


In [8]:
num_features = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 1),
    nn.Sigmoid()
)


In [9]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == "cuda":
    print("GPU detected! Training will run on GPU.")
    print("GPU name:", torch.cuda.get_device_name(0))
else:
    print("GPU not found. Training will run on CPU.")

model.to(device)

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=config.LEARNING_RATE)



GPU detected! Training will run on GPU.
GPU name: NVIDIA GeForce RTX 3050 4GB Laptop GPU


In [10]:
print(next(model.parameters()).device)


cuda:0


In [11]:
for epoch in range(config.EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{config.EPOCHS}] - Loss: {running_loss:.4f}")
torch.save(model.state_dict(), config.MODEL_SAVE_PATH)
print(f"✅ Model saved at {config.MODEL_SAVE_PATH}")


Epoch [1/10] - Loss: 1933.1730
Epoch [2/10] - Loss: 1744.1537
Epoch [3/10] - Loss: 1671.6535
Epoch [4/10] - Loss: 1625.0420
Epoch [5/10] - Loss: 1591.3616
Epoch [6/10] - Loss: 1567.2909
Epoch [7/10] - Loss: 1549.7010
Epoch [8/10] - Loss: 1524.1636
Epoch [9/10] - Loss: 1516.7233
Epoch [10/10] - Loss: 1499.4165
✅ Model saved at resnet50_dyslexia.pth


In [11]:
# Loading previously trained model
model.load_state_dict(torch.load(config.MODEL_SAVE_PATH))
print("Loaded trained model for fine-tuning")


Loaded trained model for fine-tuning


In [12]:
# Freeze all layers first
for param in model.parameters():
    param.requires_grad = False

# Unfreeze last ResNet block layer 4
for param in model.layer4.parameters():
    param.requires_grad = True

# keeping classifier trainable
for param in model.fc.parameters():
    param.requires_grad = True

print("Fine-tuning layer4 and classifier")


Fine-tuning layer4 and classifier


In [13]:
#Prevents from destroying learned features and allows the slow,stable learning
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-5
)


In [15]:
#Training loop for fine tuning
for epoch in range(config.FINETUNE_EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"[Fine-tune] Epoch {epoch+1}/{config.FINETUNE_EPOCHS} - Loss: {running_loss:.4f}")
torch.save(model.state_dict(), config.FINETUNED_MODEL_PATH)
print(f"Fine-tuned model saved at {config.FINETUNED_MODEL_PATH}")


[Fine-tune] Epoch 1/5 - Loss: 2024.6993
[Fine-tune] Epoch 2/5 - Loss: 1841.7240
[Fine-tune] Epoch 3/5 - Loss: 1723.7662
[Fine-tune] Epoch 4/5 - Loss: 1659.3012
[Fine-tune] Epoch 5/5 - Loss: 1604.8668
Fine-tuned model saved at resnet50_dyslexia_finetuned.pth
